# Proyecto 1
## Detección de ataques

In [1]:
import os.path

import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [2]:
part_1 = pd.read_csv("dataset/dataset-part1.csv")
part_2 = pd.read_csv("dataset/dataset-part2.csv")

In [3]:
clean_df = pd.concat([part_1, part_2], ignore_index=True)

## Análisis Exploratorio

In [4]:
clean_df.columns.values

array(['BIFLOW_DIRECTION', 'DIRECTION', 'DST_TO_SRC_SECOND_BYTES',
       'FIREWALL_EVENT', 'FIRST_SWITCHED', 'FLOW_ACTIVE_TIMEOUT',
       'FLOW_DURATION_MICROSECONDS', 'FLOW_DURATION_MILLISECONDS',
       'FLOW_END_MILLISECONDS', 'FLOW_END_SEC', 'FLOW_ID',
       'FLOW_INACTIVE_TIMEOUT', 'FLOW_START_MILLISECONDS',
       'FLOW_START_SEC', 'FRAME_LENGTH', 'IN_BYTES', 'IN_PKTS',
       'IPV4_DST_ADDR', 'IPV4_SRC_ADDR', 'L4_DST_PORT', 'L4_SRC_PORT',
       'LAST_SWITCHED', 'MAX_IP_PKT_LEN', 'MIN_IP_PKT_LEN',
       'OOORDER_IN_PKTS', 'OOORDER_OUT_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'PROTOCOL', 'PROTOCOL_MAP', 'RETRANSMITTED_IN_BYTES',
       'RETRANSMITTED_IN_PKTS', 'RETRANSMITTED_OUT_BYTES',
       'RETRANSMITTED_OUT_PKTS', 'SRC_TO_DST_SECOND_BYTES', 'TCP_FLAGS',
       'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT', 'TCP_WIN_MIN_IN',
       'TCP_WIN_MIN_OUT', 'TCP_WIN_MSS_IN', 'TCP_WIN_MSS_OUT',
       'TCP_WIN_SCALE_IN', 'TCP_WIN_SCALE_OUT', 'SRC_TOS', 'DST_TOS',
       'L7_PROTO_NAME', 'SAM

In [5]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12207873 entries, 0 to 12207872
Data columns (total 50 columns):
 #   Column                      Dtype 
---  ------                      ----- 
 0   BIFLOW_DIRECTION            int64 
 1   DIRECTION                   int64 
 2   DST_TO_SRC_SECOND_BYTES     object
 3   FIREWALL_EVENT              int64 
 4   FIRST_SWITCHED              int64 
 5   FLOW_ACTIVE_TIMEOUT         int64 
 6   FLOW_DURATION_MICROSECONDS  int64 
 7   FLOW_DURATION_MILLISECONDS  int64 
 8   FLOW_END_MILLISECONDS       int64 
 9   FLOW_END_SEC                int64 
 10  FLOW_ID                     int64 
 11  FLOW_INACTIVE_TIMEOUT       int64 
 12  FLOW_START_MILLISECONDS     int64 
 13  FLOW_START_SEC              int64 
 14  FRAME_LENGTH                int64 
 15  IN_BYTES                    int64 
 16  IN_PKTS                     int64 
 17  IPV4_DST_ADDR               object
 18  IPV4_SRC_ADDR               object
 19  L4_DST_PORT                 int64 
 20  

In [6]:
clean_df.head()

,BIFLOW_DIRECTION,DIRECTION,DST_TO_SRC_SECOND_BYTES,FIREWALL_EVENT,FIRST_SWITCHED,FLOW_ACTIVE_TIMEOUT,FLOW_DURATION_MICROSECONDS,FLOW_DURATION_MILLISECONDS,FLOW_END_MILLISECONDS,FLOW_END_SEC,...,TCP_WIN_MSS_IN,TCP_WIN_MSS_OUT,TCP_WIN_SCALE_IN,TCP_WIN_SCALE_OUT,SRC_TOS,DST_TOS,L7_PROTO_NAME,SAMPLING_INTERVAL,TOTAL_FLOWS_EXP,LABEL
0,1,0,40,0,1616660040,120,339,0,1616660040010,1616660040,...,1460,0,0,0,0,0,Unknown,1,2293398,Normal flow
1,1,0,",",0,1616660040,120,0,0,1616660040068,1616660040,...,0,0,0,0,40,0,ICMP,1,2293400,Normal flow
2,1,0,104,0,1616660040,120,44725,44,1616660040114,1616660040,...,0,0,0,0,0,0,TLS,1,2293404,Normal flow
3,1,0,",",0,1616660040,120,0,0,1616660040122,1616660040,...,1440,0,8,0,40,0,Unknown,1,2293407,Normal flow
4,1,0,40,0,1616660040,120,1114,1,1616660040184,1616660040,...,0,0,0,0,0,0,TLS,1,2293409,Normal flow


In [7]:
clean_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
BIFLOW_DIRECTION,12207873.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
DIRECTION,12207873.0,2.158525e-03,4.640976e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
FIREWALL_EVENT,12207873.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
FIRST_SWITCHED,12207873.0,1.617770e+09,7.090651e+05,1.616660e+09,1.616673e+09,1.618222e+09,1.618232e+09,1.618262e+09
FLOW_ACTIVE_TIMEOUT,12207873.0,1.200000e+02,0.000000e+00,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02,1.200000e+02
FLOW_DURATION_MICROSECONDS,12207873.0,2.162737e+07,3.997485e+07,0.000000e+00,4.610000e+02,2.998000e+04,1.373936e+07,1.199999e+08
FLOW_DURATION_MILLISECONDS,12207873.0,2.162695e+04,3.997481e+04,0.000000e+00,0.000000e+00,2.900000e+01,1.373900e+04,1.199990e+05
FLOW_END_MILLISECONDS,12207873.0,1.617770e+12,7.090782e+08,1.616660e+12,1.616673e+12,1.618222e+12,1.618232e+12,1.618263e+12
FLOW_END_SEC,12207873.0,1.617770e+09,7.090782e+05,1.616660e+09,1.616673e+09,1.618222e+09,1.618232e+09,1.618263e+09
FLOW_ID,12207873.0,5.890479e+07,3.531791e+07,2.293398e+06,5.345536e+06,7.978476e+07,8.283673e+07,8.588870e+07


In [8]:
clea_df.describe(include=['object'])

NameError: name 'clea_df' is not defined

In [ ]:
if not os.path.isfile("cleanEDA.html"):
    profile = ProfileReport(clean_df, title="Exploratory Data Analysis - Clean DF", minimal=True)
    profile.to_file("cleanEDA.html")

## Preprocessing

In [ ]:
# CONSTANTS 
constants = [
    'BIFLOW_DIRECTION',
    'FIREWALL_EVENT',
    'FLOW_ACTIVE_TIMEOUT',
    'FLOW_INACTIVE_TIMEOUT',
    'FRAME_LENGTH',
    'MAX_IP_PKT_LEN',
    'MIN_IP_PKT_LEN',
    'SAMPLING_INTERVAL',
    'DIRECTION',
    'OOORDER_IN_PKTS',
    'OOORDER_OUT_PKTS',
]

# DIRECTION, OOORDER_IN_PKTS and OOORDER_OUT_PKTS has 99%> zero values, so it will be considered a constant.

In [ ]:
# Drop constants from dataframe
df = clean_df.drop(constants, axis=1)
len(df.columns.values)

39

In [ ]:
# UNIQUE
unique_vars = ['FLOW_ID', 'TOTAL_FLOWS_EXP']

In [ ]:
df = df.drop(unique_vars, axis=1)
len(df.columns.values)

37

In [ ]:
if not os.path.isfile("EDA2.html"):
    profile = ProfileReport(df, title="Exploratory Data Analysis 2")
    profile.to_file("EDA2.html")

Summarize dataset:  24%|██▍       | 10/42 [02:40<08:33, 16.05s/it, Describe variable:PROTOCOL_MAP]             


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices